# Use O2C_Template Customized Notebook Template

In [2]:
import pandas as pd
import numpy as np

In [3]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [4]:
#table_name = 'ORDER_TO_CASH_ENRICHED'
table_name = 'ORDER_TO_CASH_MASTER'

sf_df = my_session.sql("select * from {}".format(table_name))
df = sf_df.to_pandas()

In [5]:
df_train = df[df['INVOICESTATUS'].isin(['LATE_PAYMENT','ONTIME_PAYMENT'])]
df_test = df[~df['INVOICESTATUS'].isin(['LATE_PAYMENT','ONTIME_PAYMENT'])]

In [6]:
df_train.shape, df_test.shape

((170445, 40), (2992, 40))

In [7]:
# Assuming df is your dataframe
df_train['ORDERDATE'] = pd.to_datetime(df_train['ORDERDATE'])
df_train['DELIVERYDATE'] = pd.to_datetime(df_train['DELIVERYDATE'])
df_train['INVOICEDATE'] = pd.to_datetime(df_train['INVOICEDATE'])
df_train['PAYMENTDATE'] = pd.to_datetime(df_train['PAYMENTDATE'])
df_train['INVOICEDUEDATE'] = pd.to_datetime(df_train['INVOICEDUEDATE'])

In [8]:
df_train['DELIVEREDON'] = pd.to_datetime(df_train['DELIVEREDON'])

In [9]:
df_train['SHIPMENTDATE'] = pd.to_datetime(df_train['SHIPMENTDATE'])

In [49]:
# Order Processing Time
df_train['SP_ORDER_PROCESSING_TIME'] = (df_train['SHIPMENTDATE'] - df_train['ORDERDATE']).dt.days

# Order Value per Unit
df_train['SP_ORDER_VALUE_PER_UNIT'] = df_train['ORDERVALUE'] / df_train['ORDERQUANTITY']

# Delivery Delay
df_train['SP_DELIVERY_DELAY'] = (df_train['DELIVEREDON'] - df_train['DELIVERYDATE']).dt.days

In [52]:
# Supplier Lifetime Value (CLV)
clv = df_train.groupby('SUPPLIERID')['ORDERVALUE'].sum().reset_index()
clv.columns = ['SUPPLIERID', 'SP_CUSTOMER_LIFETIME_VALUE']
df_train = df_train.merge(clv, on='SUPPLIERID', how='left')

In [53]:
# Order Frequency
order_freq = df_train.groupby('SUPPLIERID')['ORDERID'].nunique().reset_index()
order_freq.columns = ['SUPPLIERID', 'SP_ORDER_FREQUENCY']
df_train = df_train.merge(order_freq, on='SUPPLIERID', how='left')

In [54]:
# Average Order Value
avg_order_value = df_train.groupby('SUPPLIERID')['ORDERVALUE'].mean().reset_index()
avg_order_value.columns = ['SUPPLIERID', 'SP_AVERAGE_ORDER_VALUE']
df_train = df_train.merge(avg_order_value, on='SUPPLIERID', how='left')

In [55]:
# Recency
latest_order_date = df_train.groupby('SUPPLIERID')['ORDERDATE'].max().reset_index()
latest_order_date.columns = ['SUPPLIERID', 'SP_LAST_ORDER_DATE']
latest_order_date['SP_RECENCY'] = (pd.to_datetime('today') - latest_order_date['S_LAST_ORDER_DATE']).dt.days
df_train = df_train.merge(latest_order_date[['CUSTOMERID', 'SP_RECENCY']], on='CUSTOMERID', how='left')

In [56]:
# Preferred Payment Method
preferred_payment_method = df_train.groupby('CUSTOMERID')['PAYMENTMETHOD'].agg(lambda x: x.value_counts().index[0]).reset_index()
preferred_payment_method.columns = ['CUSTOMERID', 'CC_PREFERRED_PAYMENT_METHOD']
df_train = df_train.merge(preferred_payment_method, on='CUSTOMERID', how='left')

In [57]:
# Preferred Product Category
preferred_product_category = df_train.groupby('CUSTOMERID')['PRODUCTCATEGORY'].agg(lambda x: x.value_counts().index[0]).reset_index()
preferred_product_category.columns = ['CUSTOMERID', 'CC_PREFERRED_PRODUCT_CATEGORY']
df_train = df_train.merge(preferred_product_category, on='CUSTOMERID', how='left')

In [58]:
# Preferred Product Category
preferred_product_type = df_train.groupby('CUSTOMERID')['PRODUCTTYPE'].agg(lambda x: x.value_counts().index[0]).reset_index()
preferred_product_type.columns = ['CUSTOMERID', 'CC_PREFERRED_PRODUCT_TYPE']
df_train = df_train.merge(preferred_product_type, on='CUSTOMERID', how='left')

In [59]:
df_train.columns

Index(['ORDERID', 'ORDERITEMID', 'PRODUCTID', 'PRODUCTNAME', 'ORDERQUANTITY',
       'UNITPRICE', 'ORDERVALUE', 'CUSTOMERID', 'CUSTOMERNAME',
       'PRODUCTCATEGORY', 'ORDERDATE', 'ORDERSTATUS', 'ORDERAMOUNT',
       'ORDERCOUNT', 'SHIPMENTID', 'SHIPMENTDATE', 'DELIVERYDATE',
       'DELIVEREDON', 'CARRIER', 'SHIPMENTSTATUS', 'INVOICEID', 'INVOICEDATE',
       'INVOICEDUEDATE', 'PAYMENTDATE', 'INVOICESTATUS', 'PAYMENTMETHOD',
       'PRODUCTTYPE', 'QUANTITY', 'COMPANYTYPE', 'CONTACTDETAILS',
       'EMAILDETAILS', 'ADDRESSDETAILS', 'ADMINDETAILS', 'CREDITLIMIT',
       'CUSTOMERSINCE', 'PAYMENTTERMS', 'CREDITLIMITTYPE', 'CUSTOMERTYPE',
       'SUPPLIERNAME', 'SUPPLIERID', 'OR_ORDER_PROCESSING_TIME',
       'OR_INVOICE_PROCESSING_TIME', 'OR_ORDER_VALUE_PER_UNIT',
       'OR_DELIVERY_DELAY', 'OR_PAYMENT_DELAY', 'CC_CUSTOMER_LIFETIME_VALUE',
       'CC_ORDER_FREQUENCY', 'CC_AVERAGE_ORDER_VALUE', 'CC_RECENCY',
       'CC_PREFERRED_PAYMENT_METHOD', 'CC_PREFERRED_PRODUCT_CATEGORY',
       '

In [60]:
df_train.tail(2)

,ORDERID,ORDERITEMID,PRODUCTID,PRODUCTNAME,ORDERQUANTITY,UNITPRICE,ORDERVALUE,CUSTOMERID,CUSTOMERNAME,PRODUCTCATEGORY,...,OR_ORDER_VALUE_PER_UNIT,OR_DELIVERY_DELAY,OR_PAYMENT_DELAY,CC_CUSTOMER_LIFETIME_VALUE,CC_ORDER_FREQUENCY,CC_AVERAGE_ORDER_VALUE,CC_RECENCY,CC_PREFERRED_PAYMENT_METHOD,CC_PREFERRED_PRODUCT_CATEGORY,CC_PREFERRED_PRODUCT_TYPE
170443,OR-9f993754-37ea-4bb4-80db-4690c5a718fd,OI-339dfd1a-3443-48d2-91d7-cd738e47cc3b,PID-ed53f81f-632c-4276-a84f-3eadf889e9c2,Transmission Fluid,1603,8,12824,CID-395c2591-bd5a-4a89-a15c-f8f299c6ac78,Visionary,Automotive,...,8.0,1,-2,30421475,554,27456.204874,-93,Checks,Automotive,Radiator Coolant
170444,OR-9f993754-37ea-4bb4-80db-4690c5a718fd,OI-3b8215d7-f58d-4b5a-b6eb-d651127f88c0,PID-afb78ac2-facb-44d7-8b49-7abb2bc92639,Jumper Cables,815,13,10595,CID-395c2591-bd5a-4a89-a15c-f8f299c6ac78,Visionary,Automotive,...,13.0,1,-2,30421475,554,27456.204874,-93,Checks,Automotive,Radiator Coolant


In [61]:
# Average Order Processing Time
avg_order_processing_time = df_train.groupby('CUSTOMERID')['OR_ORDER_PROCESSING_TIME'].mean().reset_index()
avg_order_processing_time.columns = ['CUSTOMERID', 'CC_AVG_ORDER_PROCESSING_TIME']

In [62]:
# Average Invoice Processing Time
avg_invoice_processing_time = df_train.groupby('CUSTOMERID')['OR_INVOICE_PROCESSING_TIME'].mean().reset_index()
avg_invoice_processing_time.columns = ['CUSTOMERID', 'CC_AVG_INVOICE_PROCESSING_TIME']

In [63]:
# Average Delivery Delay
avg_delivery_delay = df_train.groupby('CUSTOMERID')['OR_DELIVERY_DELAY'].mean().reset_index()
avg_delivery_delay.columns = ['CUSTOMERID', 'CC_AVG_DELIVERY_DELAY']

In [64]:
# Average Payment Delay
avg_payment_delay = df_train.groupby('CUSTOMERID')['OR_PAYMENT_DELAY'].mean().reset_index()
avg_payment_delay.columns = ['CUSTOMERID', 'CC_AVG_PAYMENT_DELAY']

In [65]:
df.columns

Index(['ORDERID', 'ORDERITEMID', 'PRODUCTID', 'PRODUCTNAME', 'ORDERQUANTITY',
       'UNITPRICE', 'ORDERVALUE', 'CUSTOMERID', 'CUSTOMERNAME',
       'PRODUCTCATEGORY', 'ORDERDATE', 'ORDERSTATUS', 'ORDERAMOUNT',
       'ORDERCOUNT', 'SHIPMENTID', 'SHIPMENTDATE', 'DELIVERYDATE',
       'DELIVEREDON', 'CARRIER', 'SHIPMENTSTATUS', 'INVOICEID', 'INVOICEDATE',
       'INVOICEDUEDATE', 'PAYMENTDATE', 'INVOICESTATUS', 'PAYMENTMETHOD',
       'PRODUCTTYPE', 'QUANTITY', 'COMPANYTYPE', 'CONTACTDETAILS',
       'EMAILDETAILS', 'ADDRESSDETAILS', 'ADMINDETAILS', 'CREDITLIMIT',
       'CUSTOMERSINCE', 'PAYMENTTERMS', 'CREDITLIMITTYPE', 'CUSTOMERTYPE',
       'SUPPLIERNAME', 'SUPPLIERID'],
      dtype='object')

In [66]:
# Total Orders
total_orders = df_train.groupby('CUSTOMERID')['OR_ORDER_PROCESSING_TIME'].count().reset_index()
total_orders.columns = ['CUSTOMERID', 'CC_TOTAL_ORDERS']

In [67]:
# Total Delays
total_delays = df_train.groupby('CUSTOMERID')[['OR_DELIVERY_DELAY', 'OR_PAYMENT_DELAY']].sum().reset_index()
total_delays['CC_TOTAL_DELAYS'] = total_delays['OR_DELIVERY_DELAY'] + total_delays['OR_PAYMENT_DELAY']
total_delays = total_delays[['CUSTOMERID', 'CC_TOTAL_DELAYS']]

In [68]:
# Order Consistency
order_consistency = df_train.groupby('CUSTOMERID')['OR_ORDER_PROCESSING_TIME'].std().reset_index()
order_consistency.columns = ['CUSTOMERID', 'CC_ORDER_CONSISTENCY']

In [69]:
# Invoice Consistency
invoice_consistency = df_train.groupby('CUSTOMERID')['OR_INVOICE_PROCESSING_TIME'].std().reset_index()
invoice_consistency.columns = ['CUSTOMERID', 'CC_INVOICE_CONSISTENCY']

In [70]:
# Delivery Consistency
delivery_consistency = df_train.groupby('CUSTOMERID')['OR_DELIVERY_DELAY'].std().reset_index()
delivery_consistency.columns = ['CUSTOMERID', 'CC_DELIVERY_CONSISTENCY']

In [71]:
# Payment Consistency
payment_consistency = df_train.groupby('CUSTOMERID')['OR_PAYMENT_DELAY'].std().reset_index()
payment_consistency.columns = ['CUSTOMERID', 'CC_PAYMENT_CONSISTENCY']

In [72]:
# Merging all features into a single dataframe
features = [avg_order_processing_time, avg_invoice_processing_time, avg_delivery_delay, avg_payment_delay, total_orders,total_delays, order_consistency, invoice_consistency, delivery_consistency, payment_consistency]
customer_features = df[['CUSTOMERID']].drop_duplicates().reset_index(drop=True)

In [73]:
for feature in features:
    customer_features = customer_features.merge(feature, on='CUSTOMERID', how='left')

In [74]:
# Merge the new features back into the original dataframe
df_train = df_train.merge(customer_features, on='CUSTOMERID', how='left')

In [75]:
df_train.columns

Index(['ORDERID', 'ORDERITEMID', 'PRODUCTID', 'PRODUCTNAME', 'ORDERQUANTITY',
       'UNITPRICE', 'ORDERVALUE', 'CUSTOMERID', 'CUSTOMERNAME',
       'PRODUCTCATEGORY', 'ORDERDATE', 'ORDERSTATUS', 'ORDERAMOUNT',
       'ORDERCOUNT', 'SHIPMENTID', 'SHIPMENTDATE', 'DELIVERYDATE',
       'DELIVEREDON', 'CARRIER', 'SHIPMENTSTATUS', 'INVOICEID', 'INVOICEDATE',
       'INVOICEDUEDATE', 'PAYMENTDATE', 'INVOICESTATUS', 'PAYMENTMETHOD',
       'PRODUCTTYPE', 'QUANTITY', 'COMPANYTYPE', 'CONTACTDETAILS',
       'EMAILDETAILS', 'ADDRESSDETAILS', 'ADMINDETAILS', 'CREDITLIMIT',
       'CUSTOMERSINCE', 'PAYMENTTERMS', 'CREDITLIMITTYPE', 'CUSTOMERTYPE',
       'SUPPLIERNAME', 'SUPPLIERID', 'OR_ORDER_PROCESSING_TIME',
       'OR_INVOICE_PROCESSING_TIME', 'OR_ORDER_VALUE_PER_UNIT',
       'OR_DELIVERY_DELAY', 'OR_PAYMENT_DELAY', 'CC_CUSTOMER_LIFETIME_VALUE',
       'CC_ORDER_FREQUENCY', 'CC_AVERAGE_ORDER_VALUE', 'CC_RECENCY',
       'CC_PREFERRED_PAYMENT_METHOD', 'CC_PREFERRED_PRODUCT_CATEGORY',
       '

In [76]:
df = df_train[['CUSTOMERID', 'CUSTOMERNAME', 'CUSTOMERTYPE','COMPANYTYPE',
               'CUSTOMERSINCE', 'PAYMENTTERMS', 'CREDITLIMITTYPE',
               'CONTACTDETAILS','EMAILDETAILS','ADDRESSDETAILS','ADMINDETAILS','CREDITLIMIT', 
               'CC_CUSTOMER_LIFETIME_VALUE','CC_ORDER_FREQUENCY', 'CC_AVERAGE_ORDER_VALUE', 'CC_RECENCY',
               'CC_PREFERRED_PAYMENT_METHOD', 'CC_PREFERRED_PRODUCT_CATEGORY',
               'CC_PREFERRED_PRODUCT_TYPE', 'CC_AVG_ORDER_PROCESSING_TIME',
               'CC_AVG_INVOICE_PROCESSING_TIME', 'CC_AVG_DELIVERY_DELAY',
               'CC_AVG_PAYMENT_DELAY', 'CC_TOTAL_ORDERS', 'CC_TOTAL_DELAYS',
               'CC_ORDER_CONSISTENCY', 'CC_INVOICE_CONSISTENCY',
               'CC_DELIVERY_CONSISTENCY', 'CC_PAYMENT_CONSISTENCY']]

In [77]:
df.shape

(170445, 29)

In [78]:
df = df.drop_duplicates()

In [79]:
df.shape

(150, 29)

In [80]:
df.head(7)

,CUSTOMERID,CUSTOMERNAME,CUSTOMERTYPE,COMPANYTYPE,CUSTOMERSINCE,PAYMENTTERMS,CREDITLIMITTYPE,CONTACTDETAILS,EMAILDETAILS,ADDRESSDETAILS,...,CC_AVG_ORDER_PROCESSING_TIME,CC_AVG_INVOICE_PROCESSING_TIME,CC_AVG_DELIVERY_DELAY,CC_AVG_PAYMENT_DELAY,CC_TOTAL_ORDERS,CC_TOTAL_DELAYS,CC_ORDER_CONSISTENCY,CC_INVOICE_CONSISTENCY,CC_DELIVERY_CONSISTENCY,CC_PAYMENT_CONSISTENCY
0,CID-c2f0a78d-6978-492f-9f80-7a45887f5ba9,OTTAVA,Food Beverages,Inc,YR-2019,Net 30 days,Low,(307)111-4194,alexandria96@ellis.com,"PSC 2299, Box 5692\nAPO AP 10704",...,4.523524,18.497861,1.495295,-6.050470,1169,-5325,0.910180,11.889411,0.500192,11.885441
3,CID-e5a83adb-53ac-497f-987a-5577cbd65db5,MetroLink,Automotive,LLC,YR-2019,Net 30 days,High,640.785.2288x68059,masonandre@roberson.biz,"1803 Rodriguez Via Suite 729\nSouth Lindsey, G...",...,12.298561,22.085432,2.985612,-2.426259,1112,622,3.311176,12.977405,1.434991,13.028942
6,CID-eac5dad7-7206-4c7e-bcd9-28a1de2f468a,Lion Mark,Food Beverages,Ltd,YR-2019,Net 60 days,Very High,034.637.2385,parkjeffrey@weaver.org,"6162 Bryant Plains Suite 585\nWest Jill, ID 95338",...,4.394022,26.237319,1.488225,-18.261775,1104,-18518,0.980129,15.496434,0.500088,15.524185
7,CID-a070e2ee-afbc-4bee-97e1-468c4676a501,BlueSky Enterprises,Food Beverages,and Sons,YR-2017,Net 60 days,Very High,(452)911-9615,reidjohnathan@wilkerson.net,"PSC 4022, Box 4888\nAPO AA 52929",...,4.536804,24.405745,1.475763,-20.114901,1114,-20764,0.979113,15.826741,0.499637,15.837543
10,CID-f45ead0a-0861-42ed-98a8-0d0d74efece8,NextGen Solutions,Office Supplies,Inc,YR-2022,Net 60 days,Very High,598-905-4143x873,thomas08@martin.com,"5516 Smith Mews\nLawrencefort, IA 70261",...,10.016627,42.057007,3.148060,-2.456849,1263,873,1.806980,18.799001,1.451627,18.765155
11,CID-cd90a15c-1a99-4b4e-944c-70ae0ba47012,NexGen,Consumer Goods,Group,YR-2018,Net 15 days,Very Low,+1-321-456-6762,znguyen@jones.org,Unit 7844 Box 5894\nDPO AA 59911,...,4.523973,23.887842,1.493151,9.370719,1168,12689,0.947340,7.841746,0.500167,7.863035
14,CID-2e368d39-4d12-41c1-bf69-a535ec888a92,Wire works,Construction Materials,LLC,YR-2018,Net 15 days,Low,418.931.4865,cynthia60@dunn-johnson.org,USNS Miller\nFPO AP 04341,...,15.001757,21.133568,3.034271,6.619508,1138,10986,1.776916,9.314107,1.438160,9.305725


In [81]:
df_train_sf=my_session.createDataFrame(
        df.values.tolist(),
        schema=df.columns.tolist())
df_train_sf.write.mode("overwrite").save_as_table("FDC_HORIZONTAL.O2C_GOLD.ORDER_TO_CASH_CUSTOMER_KPI")